In [1]:
import scipy.io

# Load the .mat file
file_path = './EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
print(data.shape)


(50, 1000, 832)


In [2]:
import numpy as np

# Function to create sequences
def create_sequences(data, timesteps_in, timesteps_out):
    X, y = [], []
    for ue in data:
        for i in range(len(ue) - timesteps_in):
            X.append(ue[i:i + timesteps_in])
            y.append(ue[i + timesteps_in])
    return np.array(X), np.array(y)

timesteps_in = 5
timesteps_out = 1

X, y = create_sequences(data, timesteps_in, timesteps_out)

# Check the shapes of the result
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (49750, 5, 832), y shape: (49750, 832)


In [3]:

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


2024-05-22 21:43:45.878396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Define the TCN model
# Define the TCN model
model = Sequential([
    TCN(input_shape=(timesteps_in, X.shape[2])),
    Dense(X.shape[2])
])

model.compile(optimizer='adam', loss='mse')


/home/rithvik/anaconda3/lib/python3.11/site-packages/tcn/tcn.py:267: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)
2024-05-22 21:43:48.359243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-22 21:43:48.370321: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-22 21:43:48.372501: I external/

AttributeError: 'tuple' object has no attribute 'as_list'

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))



In [ ]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)
print(predictions)
